In [1]:
import numpy as np
import random
import time
import pprint

from pynq import PL
from pynq import Overlay
from pynq import Clocks
from pynq import allocate


PL.reset()

ol = Overlay('./design_1.bit')

for k in ol.ip_dict:
    print(k)

Clocks.pl_clk0_mhz = 100
print(f"pl_clk0_mhz = {Clocks.pl_clk0_mhz}")

rx_dma
udp_engine_100g_ip
tx_dma
cmac_usplus_0
zynq_ultra_ps_e_0
pl_clk0_mhz = 100


In [2]:
cmac = ol.cmac_usplus_0

# Actually, you don't need to plug in the 
# cmac.internal_loopback = 1
# print(cmac.internal_loopback)

cmac.start()
cmac.getStats()

{'tx': {'packets': 0,
  'good_packets': 0,
  'bytes': 0,
  'good_bytes': 0,
  'packets_large': 0,
  'packets_small': 0,
  'bad_fcs': 0,
  'pause': 0,
  'user_pause': 0},
 'rx': {'packets': 0,
  'good_packets': 0,
  'bytes': 0,
  'good_bytes': 0,
  'packets_large': 0,
  'packets_small': 0,
  'packets_undersize': 0,
  'packets_fragmented': 0,
  'packets_oversize': 0,
  'packets_toolong': 0,
  'packets_jabber': 0,
  'bad_fcs': 0,
  'packets_bad_fcs': 0,
  'stomped_fcs': 0,
  'pause': 0,
  'user_pause': 0},
 'cycle_count': 104323519}

In [3]:
from udp_engine_control import *

############################################################################################################
# Configuring FPGA Connection
############################################################################################################
my_connection_manager_sw = connection_manager_sw()
my_udp_engine_controller = udp_engine_controller(ol.udp_engine_100g_ip, my_connection_manager_sw)

MY_DST_CONFIG_MAC   =   random.getrandbits(48)
MY_SRC_CONFIG_MAC   =   random.getrandbits(48)
MY_SRC_CONFIG_IP    =   random.getrandbits(32)
MY_SRC_CONFIG_PORT  =   random.getrandbits(16)


my_udp_engine_controller.configure(MY_SRC_CONFIG_MAC, MY_SRC_CONFIG_IP, MY_SRC_CONFIG_PORT, MY_DST_CONFIG_MAC)
print("INFO:     Configured UDP Engine with:")
print(f"MY_DST_CONFIG_MAC   = {hex(MY_DST_CONFIG_MAC)}")
print(f"MY_SRC_CONFIG_MAC   = {hex(MY_SRC_CONFIG_MAC)}")
print(f"MY_SRC_CONFIG_IP    = {hex(MY_SRC_CONFIG_IP)}")
print(f"MY_SRC_CONFIG_PORT  = {hex(MY_SRC_CONFIG_PORT)}")


INFO:     Configured UDP Engine with:
MY_DST_CONFIG_MAC   = 0x48759291403f
MY_SRC_CONFIG_MAC   = 0x9a97955a9ff
MY_SRC_CONFIG_IP    = 0xd9ffc6fa
MY_SRC_CONFIG_PORT  = 0x5e7f


In [4]:
############################################################################################################
# Filling Connection Manager
############################################################################################################

COLLISION_POOL = list(my_connection_manager_sw.generate_collision_entries())
print("SUCCESS:    Generated COLLISION_POOL")

NUM_WRITES     = 100
for _ in range(NUM_WRITES):
    pick = random.choice(COLLISION_POOL)
    bind = (random.random() < 0.9)

    if bind:
        if my_udp_engine_controller.bind_connection(pick['ip'], pick['port']):
            print(f"SUCCESS:  (bind)   ipAddr = {hex(pick['ip'])}, port = {hex(pick['port'])}")
    else:
        if my_udp_engine_controller.unbind_connection(pick['ip'], pick['port']):
            print(f"SUCCESS:  (unbind) ipAddr = {hex(pick['ip'])}, port = {hex(pick['port'])}")



SUCCESS:    Generated COLLISION_POOL
SUCCESS:  (bind)   ipAddr = 0x569d067c, port = 0x196a
SUCCESS:  (bind)   ipAddr = 0x9b139657, port = 0xaf86
SUCCESS:  (bind)   ipAddr = 0xa3b8aefc, port = 0x8af0
SUCCESS:  (bind)   ipAddr = 0x426c128d, port = 0x8e6b
SUCCESS:  (bind)   ipAddr = 0x707a209b, port = 0xf721
SUCCESS:  (bind)   ipAddr = 0x3cf7d2b0, port = 0xcda8
SUCCESS:  (bind)   ipAddr = 0xc8b8c5fc, port = 0xfa8d
SUCCESS:  (bind)   ipAddr = 0xcca722e0, port = 0x7677
SUCCESS:  (bind)   ipAddr = 0x569d067c, port = 0x196a
SUCCESS:  (bind)   ipAddr = 0xb547d0d8, port = 0xc99e
SUCCESS:  (bind)   ipAddr = 0xf9c9107, port = 0xaa98
SUCCESS:  (bind)   ipAddr = 0x95b198f5, port = 0xd740
SUCCESS:  (bind)   ipAddr = 0x8ad06497, port = 0xec2f
SUCCESS:  (bind)   ipAddr = 0xf9a89c37, port = 0x35c4
SUCCESS:  (bind)   ipAddr = 0x95b198f5, port = 0xd740
SUCCESS:  (bind)   ipAddr = 0x8ad06497, port = 0xec2f
SUCCESS:  (bind)   ipAddr = 0xcca722e0, port = 0x7677
SUCCESS:  (unbind) ipAddr = 0xb3c0d65f, port =

In [13]:
############################################################################################################
# Test and Single Packet
############################################################################################################

my_udp_model = udp_model(my_connection_manager_sw,
                            MY_DST_CONFIG_MAC,
                            MY_SRC_CONFIG_MAC,
                            MY_SRC_CONFIG_IP,
                            MY_SRC_CONFIG_PORT)

test_unit    = my_udp_model.generate_test()

while test_unit['dropped'] == 1:
    test_unit    = my_udp_model.generate_test()

print("SUCCESS:     Generated test unit with:")
print(f"   connection_id = {test_unit['connection_id']}")
print(f"   input_payload_bytes_length = {test_unit['input_payload_bytes_length']}")
print(f"   output_packet_bytes_length = {test_unit['output_packet_bytes_length']}")


SUCCESS:     Generated test unit with:
   connection_id = 191531
   input_payload_bytes_length = 238
   output_packet_bytes_length = 280


In [14]:
## send packet
tx_dma = ol.tx_dma

# Create DMA buffer and fill with reversed payload as uint8
dma_buf_in = allocate(test_unit['input_payload_bytes_length'], dtype=np.uint8)
dma_buf_in[:] = np.array(test_unit['input_payload_bytes'], dtype=np.uint8)

# Send the packet
tx_dma.sendchannel.transfer(dma_buf_in)
tx_dma.sendchannel.wait()
print("SUCCESS: packet sent through tx_dma")


SUCCESS: packet sent through tx_dma


In [15]:
cmac.getStats()

{'tx': {'packets': 0,
  'good_packets': 0,
  'bytes': 0,
  'good_bytes': 0,
  'packets_large': 0,
  'packets_small': 0,
  'bad_fcs': 0,
  'pause': 0,
  'user_pause': 0},
 'rx': {'packets': 0,
  'good_packets': 0,
  'bytes': 0,
  'good_bytes': 0,
  'packets_large': 0,
  'packets_small': 0,
  'packets_undersize': 0,
  'packets_fragmented': 0,
  'packets_oversize': 0,
  'packets_toolong': 0,
  'packets_jabber': 0,
  'bad_fcs': 0,
  'packets_bad_fcs': 0,
  'stomped_fcs': 0,
  'pause': 0,
  'user_pause': 0},
 'cycle_count': 29458489093}

In [ ]:
## receive actual tx packet
rx_dma = ol.rx_dma

dma_buf_out = allocate(test_unit['output_packet_bytes_length'], dtype=np.uint8)

rx_dma.recvchannel.transfer(dma_buf_out)
rx_dma.recvchannel.wait()
print("SUCCESS:    packet received through rx_dma")

In [ ]:
## compare output with expected
print(f"output matches = {dma_buf_out == test_unit['output_packet_bytes']}")
print(f"dma_buf_out = \n{dma_buf_out}\n")
print(f"output_packet_bytes = \n{test_unit['output_packet_bytes']}\n")


In [6]:
cmac.getStats()

{'tx': {'packets': 0,
  'good_packets': 0,
  'bytes': 0,
  'good_bytes': 0,
  'packets_large': 0,
  'packets_small': 0,
  'bad_fcs': 0,
  'pause': 0,
  'user_pause': 0},
 'rx': {'packets': 0,
  'good_packets': 0,
  'bytes': 0,
  'good_bytes': 0,
  'packets_large': 0,
  'packets_small': 0,
  'packets_undersize': 0,
  'packets_fragmented': 0,
  'packets_oversize': 0,
  'packets_toolong': 0,
  'packets_jabber': 0,
  'bad_fcs': 0,
  'packets_bad_fcs': 0,
  'stomped_fcs': 0,
  'pause': 0,
  'user_pause': 0},
 'cycle_count': 1198532791}